# Load data from JSON file
- Create empty dataframe
- Loop on every question/answer pair in the file
- Add a row to the dataframe for each pair

In [31]:
import json
import pandas as pd

question_df = pd.DataFrame(columns=["Question","Context","Text answer","Answer start"])

FILE_PATH = "../data/"

with open(FILE_PATH+'valid.json') as json_file:
    data = json.load(json_file)
    for book in data['data']:
        for paragraph in book['paragraphs']:
            for question in paragraph['qas']:
                row = pd.Series(data={'Question' : question['question'],
                                      'Context' : paragraph['context'],
                                      'Text answer': question['answers'][0]['answer_start'],
                                      'Answer start': question['answers'][0]['text']})
                question_df = question_df.append(row,ignore_index=True)

In [32]:
display(question_df)

,Question,Context,Text answer,Answer start
0,Que concerne principalement les documents ?,Les deux tableaux sont certes décrits par des ...,161,La Vierge aux rochers
1,Par quoi sont décrit les deux tableaux ?,Les deux tableaux sont certes décrits par des ...,46,documents contemporains
2,Quels types d'objets sont les deux tableaux au...,Les deux tableaux sont certes décrits par des ...,204,objets de spéculations
3,Sur quelle jambe les personnages se tiennent-t...,Les deux panneaux présentent de nombreuses sim...,242,droite
4,Quel pied avancent les personnages ?,Les deux panneaux présentent de nombreuses sim...,271,gauche
...,...,...,...,...
3183,A quel risque la zone où se situe la chapelle ...,La chapelle se trouvant dans une zone inondabl...,33,zone inondable
3184,Comment était disposés les murets par rapport ...,La chapelle se trouvant dans une zone inondabl...,210,perpendiculaires au courant
3185,En quelle matière sont les dalles du canal ?,La chapelle se trouvant dans une zone inondabl...,322,schiste
3186,Par quoi le canal est-il doublé ?,La chapelle se trouvant dans une zone inondabl...,479,par une digue


# Load the model

In [4]:
from gensim.models import KeyedVectors
import fasttext
# Load vectors directly from the file
def load_model():
    try:
        # model = KeyedVectors.load_word2vec_format('../model/frWac_postag_no_phrase_1000_skip_cut100.bin', binary=True)
        model = fasttext.load_model("../model/cc.fr.300.bin")
        print("model loaded")
        return model
    except:
        print("model not found")
        
model = load_model()

model loaded


# Check if a word exists in the model

In [5]:
def get_WordVector(word,model):
    try:
        vector = model[word]
        return vector
    except:
        print("Not found")

# Compute cosine distance between 2 words

In [13]:
from scipy import spatial

def get_cosine_distance(word_1,word_2,model):
    vec_word_1 = (get_WordVector(word_1,model))
    vec_word_2 = (get_WordVector(word_2,model))
    return (1 - spatial.distance.cosine(vec_word_1, vec_word_2))

print(get_cosine_distance("word","word",model))
print(get_cosine_distance("manger","pleurer",model))
print(get_cosine_distance("manger","boire",model))
print(get_cosine_distance("manger","mourir",model))

1.0
0.3769944906234741
0.6689237952232361
0.44056111574172974
